Set `autoreload` to execute the change in `.py` files.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import _pickle
from pathlib import Path
import gc
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from __utils__ import image_processing
from __utils__ import label_processing
from __utils__ import labeling
from __utils__ import feature_processing
from __utils__ import loso_preparing
from __utils__ import functions

In [ ]:
dataset_dir = "D:/Databases/CAS(ME)^2"
# dataset_dir = "I:/HEH/Databases/CAS(ME)^2"
# dataset_dir = "/data/disk1/heh/databases/CAS(ME)^2"

images_loading = False
image_size = 128
load_cropped_images = False
# expression_type = "mae"  # macro-expression spotting
expression_type = "me"  # micro-expression spotting
debug_preds = True
labeling_function = "pseudo_labeling"
# labeling_function = "original_labeling"
model_names = {
    0: "SOFTNet",
    1: "SOFTNetCBAM",
    2: "ViT-B",
    3: "SL-ViT-B",
    4: "Swin-T",
    5: "Swin-S",
    6: "L-Swin-T",
    7: "S-Swin-T",
    8: "SL-Swin-T",
    9: "SL-Swin-S",
}
model_name = model_names[8]
batch_size = 48
epochs = 25
save_preds = False
preds_stem = (
    f"{expression_type}_"
    + model_name.lower().replace("-", "_")
    + f"_batch_size_{batch_size}"
    + f"_epochs_{epochs}"
    + f"_{labeling_function}"
    + f"_{image_size}"
    + "_3"
)
preds_path = Path(dataset_dir, "preds", preds_stem).with_suffix(".pkl")
print(f"preds_path: {preds_path}")

## Load Images


When debug the image processing, the videos_images is from cropped_rawpic, whereas the other variables are from rawpic.


In [4]:
videos_images, subjects, subjects_videos_code = image_processing.load_images(
    dataset_dir,
    images_loading=images_loading,
    image_size=image_size,
    load_cropped_images=load_cropped_images,
)

subject:  s15
subject:  s16
subject:  s19
subject:  s20
subject:  s21
subject:  s22
subject:  s23
subject:  s24
subject:  s25
subject:  s26
subject:  s27
subject:  s29
subject:  s30
subject:  s31
subject:  s32
subject:  s33
subject:  s34
subject:  s35
subject:  s36
subject:  s37
subject:  s38
subject:  s40


In [5]:
print("subjects:", subjects)
print("subjects_videos_code:", subjects_videos_code)

subjects: ['s15', 's16', 's19', 's20', 's21', 's22', 's23', 's24', 's25', 's26', 's27', 's29', 's30', 's31', 's32', 's33', 's34', 's35', 's36', 's37', 's38', 's40']
subjects_videos_code: [['0101', '0102', '0401', '0402', '0502', '0503', '0505', '0508'], ['0101', '0102', '0401', '0402', '0502', '0505', '0507'], ['0102', '0402', '0505', '0507', '0502'], ['0502'], ['0101', '0401'], ['0101', '0102', '0402', '0503', '0508'], ['0102', '0402', '0503', '0507'], ['0101', '0401', '0402', '0502', '0507'], ['0101', '0102', '0502', '0508'], ['0101', '0102', '0401', '0503'], ['0101', '0102', '0401', '0402', '0502', '0503', '0505', '0507', '0508'], ['0502'], ['0101', '0102', '0401', '0502', '0503', '0505', '0507'], ['0101', '0401', '0402', '0502', '0503', '0505', '0507'], ['0101', '0102', '0401', '0402', '0502', '0503', '0505', '0507', '0508'], ['0102', '0402'], ['0401', '0402', '0503'], ['0102'], ['0401', '0505'], ['0101', '0402', '0502', '0505', '0507', '0508'], ['0502', '0507'], ['0401', '0502', '

## Load Excel 


In [7]:
Excel_data = label_processing.load_excel(dataset_dir)
Excel_data.head(5)

,participant,video_name_&_expression_number,onset,apex,offset,AUs,extimated_emotion,expression_type,self-reported_emotion,video_name,video_code,subject
0,1,anger1_1,557,572,608,4+10+14+15,negative,macro-expression,anger,anger1,0401,s15
1,1,anger1_2,2854,2862,2871,38,others,macro-expression,sadness,anger1,0401,s15
2,1,anger2_1,2155,2163,0,NaN,negative,macro-expression,anger,anger2,0402,s15
3,1,anger2_2,3363,3371,3383,4+7+14,negative,macro-expression,anger,anger2,0402,s15
4,1,anger2_3,3380,3386,3407,4+14+38,negative,macro-expression,anger,anger2,0402,s15


## Load Ground Truth Labels


In [8]:
(
    clean_videos_images,
    clean_subjects_videos_code,
    clean_subjects,
    clean_subjects_videos_ground_truth_labels,
) = label_processing.load_ground_truth_labels(
    dataset_dir,
    expression_type,
    videos_images,
    subjects_videos_code,
    subjects,
    Excel_data,
)

required_videos_index:  [1, 4, 8, 9, 12, 13, 14, 16, 28, 33, 36, 37, 38, 45, 46, 47, 49, 50, 52, 54, 55, 57, 62, 64, 67, 71, 73, 74, 77, 83, 87, 91, 93]
len(clean_videos_images) = 33


In [9]:
print("len(clean_subjects): ", len(clean_subjects))
print("clean_subjects: ", clean_subjects)
print("len(clean_subjects_videos_code): ", len(clean_subjects_videos_code))
print("clean_subjects_videos_codes: ", clean_subjects_videos_code)
print(
    "len(clean_subjects_videos_ground_truth_labels): ",
    len(clean_subjects_videos_ground_truth_labels),
)
print(
    "clean_subjects_videos_ground_truth_labels: ",
    clean_subjects_videos_ground_truth_labels,
)

len(clean_subjects):  14
clean_subjects:  ['s15' 's16' 's19' 's23' 's24' 's25' 's27' 's29' 's30' 's31' 's32' 's35'
 's37' 's38']
len(clean_subjects_videos_code):  14
clean_subjects_videos_codes:  [['0102', '0502'], ['0101', '0102', '0502', '0505', '0507'], ['0402'], ['0102'], ['0401', '0507'], ['0101', '0102'], ['0101', '0102', '0401', '0502', '0503', '0507'], ['0502'], ['0101', '0401'], ['0101', '0402', '0505'], ['0401', '0502', '0503', '0508'], ['0102'], ['0402', '0508'], ['0507']]
len(clean_subjects_videos_ground_truth_labels):  14
clean_subjects_videos_ground_truth_labels:  [[[[698, 706]], [[137, 147]]], [[[551, 564]], [[269, 277]], [[322, 333]], [[395, 406], [1694, 1709], [1879, 1894]], [[1957, 1967], [2284, 2294]]], [[[1926, 1941]]], [[[330, 345], [525, 539], [726, 739]]], [[[607, 620], [962, 976], [1889, 1901], [2180, 2192], [3440, 3452]], [[1835, 1847], [1950, 1964], [3232, 3247]]], [[[112, 126]], [[995, 1007], [1007, 1016], [1017, 1033]]], [[[873, 887]], [[33, 47], [308, 316],

## Calculate k


In [10]:
k = label_processing.calculate_k(clean_subjects_videos_ground_truth_labels)

k (Half of average length of expression) =  6


## Extract Features


In [11]:
# clean_videos_images_features = feature_processing.extract_features(
#     clean_videos_images, k, image_size=128
# )

## Pre-processing


In [12]:
# resampled_clean_videos_images_features = feature_processing.preprocess(
#     clean_videos_images, clean_videos_images_features, k
# )

### Dump Resampled Clean Videos Images Features


In [13]:
# with open(
#     Path(
#         dataset_dir,
#         f"resampled_clean_videos_images_{expression_type}_features_"
#         + str(image_size)
#         + ".pkl",
#     ),
#     "wb",
# ) as pkl_file:
#     _pickle.dump(resampled_clean_videos_images_features, pkl_file)
#     pkl_file.close()

### Load **Original** Resampled Clean Videos Images Features


In [14]:
# with open(
#     Path(
#         dataset_dir,
#         f"original_resampled_clean_videos_images_{expression_type}_features.pkl",
#     ),
#     "rb",
# ) as pkl_file:
#     resampled_clean_videos_images_features = _pickle.load(pkl_file)
#     pkl_file.close()

### Load Resampled Clean Videos Images Features


In [15]:
if debug_preds is False:
    with open(
        Path(
            dataset_dir,
            f"resampled_clean_videos_images_{expression_type}_features_"
            + str(image_size)
            + ".pkl",
        ),
        "rb",
    ) as pkl_file:
        resampled_clean_videos_images_features = _pickle.load(pkl_file)
        pkl_file.close()

In [16]:
if debug_preds is False:
    print(
        "len(resampled_clean_videos_images_features): ",
        len(resampled_clean_videos_images_features),
    )
    print(
        "len(resampled_clean_videos_images_features[0]): ",
        len(resampled_clean_videos_images_features[0]),
    )
    print(
        "resampled_clean_videos_images_features[0][0].shape: ",
        resampled_clean_videos_images_features[0][0].shape,
    )

## Labeling


In [17]:
if debug_preds is False:
    if labeling_function == "pseudo_labeling":
        labels = labeling.get_pseudo_labels(
            clean_videos_images, clean_subjects_videos_ground_truth_labels, k
        )
    elif labeling_function == "original_labeling":
        labels = labeling.get_original_labels(
            clean_videos_images, clean_subjects_videos_ground_truth_labels, k
        )

Total frames: 80463


## Prepare for LOSO


In [18]:
if debug_preds is False:
    X, y, groups = loso_preparing.legacy_prepare_for_loso(
        resampled_clean_videos_images_features,
        labels,
        clean_subjects,
        clean_videos_images,
        clean_subjects_videos_ground_truth_labels,
        k,
    )

In [19]:
if debug_preds is False:
    del resampled_clean_videos_images_features
    gc.collect()

## Training


In [25]:
if debug_preds is False:
    from __utils__.training_dev import train

    preds = train(
        X=X,
        y=y,
        groups=groups,
        expression_type=expression_type,
        model_name=model_name,
        train_or_not=True,
        epochs=epochs,
        batch_size=batch_size,
    )
else:
    with open(preds_path, "rb") as pkl_file:
        preds = _pickle.load(pkl_file)
        pkl_file.close()

In [26]:
if save_preds is True:
    with open(preds_path, "wb") as pkl_file:
        _pickle.dump(preds, pkl_file)
        pkl_file.close()

## Spotting


In [27]:
metric_fn, result_dict = functions.spot_and_evaluate(
    preds,
    clean_subjects_videos_ground_truth_labels,
    clean_videos_images,
    clean_subjects,
    clean_subjects_videos_code,
    k,
    p=0.60,
    show_plot_or_not=False,
)

Split 0 / 14 is in process.
0 video(s) have been processed.
The current video be processed: subject s15, video 0102
The current video be processed: subject s15, video 0502

True Positive: 0 False Posive: 21 False Negative: 2
Precision = 2.220446049250313e-16, Recall =2.220446049250313e-16, F1-Score = 4.440892098500626e-16
Split 0 / 14 is processed.

Split 1 / 14 is in process.
2 video(s) have been processed.
The current video be processed: subject s16, video 0101
The current video be processed: subject s16, video 0102
The current video be processed: subject s16, video 0502
The current video be processed: subject s16, video 0505
The current video be processed: subject s16, video 0507

True Positive: 1 False Posive: 53 False Negative: 9
Precision = 0.01851851851851874, Recall =0.10000000000000023, F1-Score = 0.03125000000000055
Split 1 / 14 is processed.

Split 2 / 14 is in process.
7 video(s) have been processed.
The current video be processed: subject s19, video 0402

True Positive: 1 

## Final Evaluation


In [28]:
functions.final_evaluate(metric_fn, result_dict)

True Positive: 14 False Posive: 298 False Negative: 43
COCO AP@[.5:.95]: 0.0036
Final Precision = 0.044871794871795094,
Final Recall =0.2456140350877195,
Final F1-Score = 0.07588075880758863

Highest Precision = 0.05113636363636386,
Highest Recall =0.2500000000000002,
Highest F1-Score = 0.08450704225352167


| Parameters | Value | Value | Value
| --- | --- | --- | ---
| model | 3D-CNN | SOFTNet | SOFTNet (main_dev)
| epochs | | 20 (39 m)| 20 (39 m)
| batch_size | | 48 | 48
| learning_rate | | 0.0005 | 0.0005
| True Positive | | 19 | 16
| False Positive | | 252 | 238
| False Negative | | 38 | 41
| Precision | | 0.0701 | 0.0629
| Recall | | 0.3333 | 0.2807
| F1-Score | 0.0714 | 0.1159 | 0.1028


| Parameters | Value |
| --- | --- |
| model | ViT |
| epochs | 20 (159 m) |
| batch_size | 48 |
| learning_rate | 0.0005 |
| learning_rate_decay | none |
| True Positive | 2 |
| False Positive | 776 |
| False Negative | 55 |
| Precision | 0.0025 |
| Recall | 0.0350 |
| F1-Score | 0.0047 |


without pos_embedding


| Parameters | Value | Value
| --- | --- | ---
| model | SL-Swin-T | SL-Swin-T
| epochs | 25 | 20 (665 m)
| batch_size | 32 | 96
| learning_rate | 0.0005 | 0.0005
| learning_rate_decay | none | none
| True Positive | 12 | 4
| False Positive | 403 | 366
| False Negative | 45 | 53
| Precision | 0.0289 | 0.0108
| Recall | 0.2105 | 0.0701
| F1-Score | 0.0508 | 0.0187


## Ablation Study


In [29]:
ablation_dict = functions.ablation_study_p_dev(
    preds,
    clean_subjects_videos_ground_truth_labels,
    clean_videos_images,
    clean_subjects,
    clean_subjects_videos_code,
    k,
)

 p | TP | FP | FN | Precision | Recall | F1-Score
0.01 | 41 | 4734 | 16 | 0.0086 | 0.7193 | 0.0170 |
0.02 | 41 | 4534 | 16 | 0.0090 | 0.7193 | 0.0177 |
0.03 | 39 | 4298 | 18 | 0.0090 | 0.6842 | 0.0178 |
0.04 | 39 | 4111 | 18 | 0.0094 | 0.6842 | 0.0185 |
0.05 | 37 | 3928 | 20 | 0.0093 | 0.6491 | 0.0184 |
0.06 | 37 | 3761 | 20 | 0.0097 | 0.6491 | 0.0192 |
0.07 | 37 | 3560 | 20 | 0.0103 | 0.6491 | 0.0203 |
0.08 | 37 | 3397 | 20 | 0.0108 | 0.6491 | 0.0212 |
0.09 | 37 | 3219 | 20 | 0.0114 | 0.6491 | 0.0223 |
0.10 | 37 | 3074 | 20 | 0.0119 | 0.6491 | 0.0234 |
0.11 | 35 | 2909 | 22 | 0.0119 | 0.6140 | 0.0233 |
0.12 | 34 | 2768 | 23 | 0.0121 | 0.5965 | 0.0238 |
0.13 | 34 | 2654 | 23 | 0.0126 | 0.5965 | 0.0248 |
0.14 | 33 | 2536 | 24 | 0.0128 | 0.5789 | 0.0251 |
0.15 | 33 | 2405 | 24 | 0.0135 | 0.5789 | 0.0265 |
0.16 | 32 | 2282 | 25 | 0.0138 | 0.5614 | 0.0270 |
0.17 | 29 | 2183 | 28 | 0.0131 | 0.5088 | 0.0256 |
0.18 | 28 | 2078 | 29 | 0.0133 | 0.4912 | 0.0259 |
0.19 | 27 | 1975 | 30 | 0.0135 |